In [18]:
import pandas as pd
import numpy as np
import math

In [20]:
event_training = pd.read_csv('training.csv')
event_test = pd.read_csv('test.csv')
training = pd.read_csv('labels_training_set.csv')
test =  pd.read_csv('trocafone_kaggle_test.csv')
paso_5_train = pd.read_csv('train_paso_5.csv')
paso_5_test = pd.read_csv('test_paso_5.csv')

# vamos a realizar un mean encoding segun Pais,region,ciudad, tipo de dispositivo, sistema operativo de busqueda y version del browser

# primero para los usuarios del set de entrenamiento

In [21]:
#primero hago el mean encoding de ciudad, country y region, para eso defino la siguiente funcion:

In [22]:
def encodearTrain(dataframe ,columna , evento , nombre):
    label_count = dataframe.loc[dataframe['event']==evento,[columna,'person','label']]
    count = dataframe.loc[dataframe['event']==evento,[columna,'person']]\
                    .groupby(columna).count().reset_index()
    count.columns=[columna,'count']
    label_count = label_count.groupby([columna,'label']).count().unstack().reset_index().fillna(0)
    label_count.columns=[columna,'label_0_count','label_1_count']
    tabla = label_count.drop('label_0_count',axis=1).merge(count,on=columna,how='inner')
    tabla[nombre] = tabla['label_1_count'] / tabla['count']
    tabla = tabla.drop(['label_1_count','count'],axis=1)
    usuarios_con_evento = tabla.merge(dataframe,on=columna,how='inner').loc[:,['person',nombre]]\
                            .groupby('person').max().reset_index()
    usuarios_sin_evento = dataframe.loc[:,['person','event']].groupby('person')\
                                        .filter(lambda x: (x['event']!=evento).all())\
                                            .reset_index(drop=True).drop_duplicates(subset='person',keep='first')
    usuarios_sin_evento.columns=['person',nombre]
    usuarios_sin_evento[nombre]=-1
    feature = usuarios_sin_evento.merge(usuarios_con_evento,on=['person',nombre],how='outer')
    
    return feature, tabla

In [23]:
mean_city, tabla_city = encodearTrain(event_training,'city','visited site','city_mean')
mean_region, tabla_region = encodearTrain(event_training,'region','visited site','region_mean')
mean_country, tabla_country = encodearTrain(event_training,'country','visited site','country_mean')
mean_device,tabla_device = encodearTrain(event_training,'device_type','visited site','device_mean')

In [24]:
feature_train = training.merge(mean_city,on='person',how='inner').merge(mean_region,on='person',how='inner')\
                    .merge(mean_country,on='person',how='inner').merge(mean_device,on='person',how='inner')\
                          .merge(paso_5_train,on=['person','label'],how='inner')\

In [25]:
feature_train.head()

,person,label,city_mean,region_mean,country_mean,device_mean,most_frequent_event_0,most_frequent_event_1,most_frequent_event_2,most_frequent_event_3,...,Sony,iPad,iPhone,Direct,Email,Organic,Paid,Referral,Social,desconocido_channel
0,0566e9c1,0,0.517241,0.081024,0.088292,0.073837,0,0,0,0,...,1.0,0.0,11.0,8.0,0.0,1.0,0.0,8.0,0.0,0.0
1,6ec7ee77,0,0.086865,0.085382,0.088292,0.073837,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,abe7a2fb,0,0.096139,0.081897,0.088292,0.073837,0,0,0,0,...,0.0,0.0,18.0,2.0,0.0,2.0,6.0,10.0,2.0,0.0
3,34728364,0,0.121673,0.081897,0.088292,0.073837,0,0,0,0,...,3.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0
4,87ed62de,0,0.185022,0.111850,0.088292,0.102718,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# Ahora repito para el set de prediccion, uso las tablas calculadas en la seccion anterior:

In [26]:
# defino la siguiente funcion:

In [27]:
def encodearTest1(dataframe, columna,evento,nombre ,tabla):
    
    objeto = dataframe.loc[:,['person',columna]].drop_duplicates(subset=['person',columna],keep='first')
    objeto = objeto.dropna()
    objeto = objeto.merge(tabla,on=columna,how='left')
    objeto['bool'] = objeto[nombre].isnull()
    objeto_uno = objeto.loc[objeto['bool']==True,['person',nombre]]
    objeto_dos = objeto.loc[objeto['bool']==False,['person',nombre]]

    objeto_uno[nombre] = objeto.loc[objeto[columna]=='Unknown',nombre].reset_index().iloc[1]
    
    objeto_dos = pd.concat([objeto_uno,objeto_dos])
    
    objeto_dos = objeto_dos.groupby('person').max().reset_index()
    usuarios_sin_evento = dataframe.loc[:,['person','event']].groupby('person')\
                                        .filter(lambda x: (x['event']!=evento).all())\
                                            .reset_index(drop=True).drop_duplicates(subset='person',keep='first')
    usuarios_sin_evento.columns=['person',nombre]
    usuarios_sin_evento[nombre]=-1
    
    feature = pd.concat([usuarios_sin_evento,objeto_dos])
    
    return feature

In [28]:
#def encodearTest2(dataframe, columna,evento,nombre ,tabla):
    
#    objeto = dataframe.loc[:,['person',columna]].drop_duplicates(subset=['person',columna],keep='first')
#    objeto = objeto.dropna()
#    objeto = objeto.merge(tabla,on=columna,how='left')
#    objeto['bool'] = objeto[nombre].isnull()
#    objeto_uno = objeto.loc[objeto['bool']==True,['person',nombre]]
#    objeto_dos = objeto.loc[objeto['bool']==False,['person',nombre]]

#    objeto_uno[nombre] = tabla.loc[:,nombre].reset_index().mean()
    
#    objeto_dos = pd.concat([objeto_uno,objeto_dos])
    
#    usuarios_sin_evento = dataframe.loc[:,['person','event']].groupby('person')\
#                                        .filter(lambda x: (x['event']!=evento).all())\
#                                            .reset_index(drop=True).drop_duplicates(subset='person',keep='first')
#    usuarios_sin_evento.columns=['person',nombre]
#    usuarios_sin_evento[nombre]=-1
    
#    feature = pd.concat([usuarios_sin_evento,objeto_dos])
    
#   return feature

In [29]:
mean_city = encodearTest1(event_test,'city','visited site','city_mean',tabla_city)
mean_region = encodearTest1(event_test,'region','visited site','region_mean',tabla_region)
mean_country = encodearTest1(event_test,'country','visited site','country_mean',tabla_country)
mean_device = encodearTest1(event_test,'device_type','visited site','device_mean',tabla_device)

In [30]:
feature_test = test.merge(mean_city,on='person',how='inner').merge(mean_region,on='person',how='inner')\
                   .merge(mean_country,on='person',how='inner')\
                    .merge(mean_device,on='person',how='inner').merge(paso_5_test,on='person',how='inner')

In [31]:
feature_test.head()

,person,city_mean,region_mean,country_mean,device_mean,most_frequent_event_0,most_frequent_event_1,most_frequent_event_2,most_frequent_event_3,most_frequent_event_4,...,Sony,iPad,iPhone,Direct,Email,Organic,Paid,Referral,Social,desconocido_channel
0,4886f805,0.000000,0.081024,0.088292,0.073837,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0297fc1e,0.091503,0.081024,0.088292,0.073837,0,0,0,0,1,...,0.0,5.0,405.0,56.0,0.0,1.0,21.0,17.0,0.0,0.0
2,2d681dd8,0.041284,0.081897,0.088292,0.102718,0,0,0,0,1,...,0.0,0.0,6.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,cccea85e,0.046832,0.081897,0.088292,0.102718,0,0,0,0,1,...,22.0,0.0,8.0,1.0,0.0,14.0,6.0,1.0,0.0,0.0
4,4c8a8b93,0.017794,0.081897,0.088292,0.073837,0,0,0,0,1,...,0.0,0.0,176.0,2.0,0.0,3.0,6.0,9.0,0.0,0.0


In [32]:
feature_train.head()

,person,label,city_mean,region_mean,country_mean,device_mean,most_frequent_event_0,most_frequent_event_1,most_frequent_event_2,most_frequent_event_3,...,Sony,iPad,iPhone,Direct,Email,Organic,Paid,Referral,Social,desconocido_channel
0,0566e9c1,0,0.517241,0.081024,0.088292,0.073837,0,0,0,0,...,1.0,0.0,11.0,8.0,0.0,1.0,0.0,8.0,0.0,0.0
1,6ec7ee77,0,0.086865,0.085382,0.088292,0.073837,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,abe7a2fb,0,0.096139,0.081897,0.088292,0.073837,0,0,0,0,...,0.0,0.0,18.0,2.0,0.0,2.0,6.0,10.0,2.0,0.0
3,34728364,0,0.121673,0.081897,0.088292,0.073837,0,0,0,0,...,3.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0
4,87ed62de,0,0.185022,0.111850,0.088292,0.102718,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [33]:
feature_train.to_csv('train_paso_7.csv',index=False)
feature_test.to_csv('test_paso_7.csv',index=False)